In [2]:
## CELL 1
import findspark
findspark.init()
import pyspark
findspark.find()

from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("Data Analysis") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.7") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g") \
    .config("spark.executor.memoryOverhead", "4g") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .config("spark.driver.maxResultSize", "8g") \
    .getOrCreate()

In [4]:
# Load the tech layoffs dataset
tech_layoffs_df = spark.read.parquet('tech_layoffs_sk.parquet')

# Load the housing dataset
housing_df = spark.read.parquet('housing_market_sk.parquet')

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/C:/Users/aryan/Documents/bigdata-project/tech_layoffs_sk.parquet.

In [4]:
# show top 5 rows
tech_layoffs_df.show(5)

+------+-----------+--------------------+---------+-------------+--------+------------+----------+---------------------------+--------------------------+--------+--------+---------------------+------+--------+----------+
|     #|    Company|         Location_HQ|  Country|    Continent|Laid_Off|Date_layoffs|Percentage|Company_Size_before_Layoffs|Company_Size_after_layoffs|Industry|   Stage|Money_Raised_in_$_mil|  Year|     lat|       lng|
+------+-----------+--------------------+---------+-------------+--------+------------+----------+---------------------------+--------------------------+--------+--------+---------------------+------+--------+----------+
|1492.0|  Springbig|               Miami|      USA|North America|    37.0|     44896.0|      23.0|                      161.0|                     124.0|   Sales|Post-IPO|                  $32|2022.0|25.77427| -80.19366|
|1498.0|   DoorDash|San Francisco Bay...|      USA|North America|  1250.0|     44895.0|       6.0|                  

In [5]:
# show top 5 rows of housing dataset
housing_df.show(5)

+---------+--------------+-------------------+-----------+------------+----------+--------+----------------+--------------------+------------------+----------------------+---------------------------------+-------------------------------------+-----------------------------------+---------------------------------------+-----------------------------------+---------------------------------------+------------------+---------------------+------------------+--------------------+--------------------+------------------------+-----------+------------------+----------------------------------------+--------------------------------------------+-------------+--------------------+-------------------+-----------------------+-----------------------+---------------------------+----------------------+--------------------------+-------------------------------+-----------------------------------+------------------------------+----------------------------------+------------------+----------------------+----

In [6]:
# Selecting relevant columns for analysis
selected_columns = [
    'region_id',
    'region_type_id',
    'region_name',
    'region_type',
]

# Displaying the data for analysis
housing_df.select(selected_columns).show()

+---------+--------------+--------------------+-----------+
|region_id|region_type_id|         region_name|region_type|
+---------+--------------+--------------------+-----------+
|     2563|             5| Davidson County, TN|     county|
|     2563|             5| Davidson County, TN|     county|
|     2980|             5|    Henry County, VA|     county|
|     2980|             5|    Henry County, VA|     county|
|     2980|             5|    Henry County, VA|     county|
|    10420|            -2|Akron, OH metro area|      metro|
|    10420|            -2|Akron, OH metro area|      metro|
|    10420|            -2|Akron, OH metro area|      metro|
|    33940|            -2|Montrose, CO metr...|      metro|
|    33940|            -2|Montrose, CO metr...|      metro|
|    33940|            -2|Montrose, CO metr...|      metro|
|    37740|            -2|Payson, AZ metro ...|      metro|
|    37740|            -2|Payson, AZ metro ...|      metro|
|    37740|            -2|Payson, AZ met

In [7]:
# show me all column names from housing
housing_df.columns

['region_id',
 'region_type_id',
 'region_name',
 'region_type',
 'period_begin',
 'period_end',
 'duration',
 'total_homes_sold',
 'total_homes_sold_yoy',
 'average_homes_sold',
 'average_homes_sold_yoy',
 'total_homes_sold_with_price_drops',
 'total_homes_sold_with_price_drops_yoy',
 'average_homes_sold_with_price_drops',
 'average_homes_sold_with_price_drops_yoy',
 'percent_homes_sold_with_price_drops',
 'percent_homes_sold_with_price_drops_yoy',
 'median_sale_price',
 'median_sale_price_yoy',
 'median_sale_ppsf',
 'median_sale_ppsf_yoy',
 'median_days_to_close',
 'median_days_to_close_yoy',
 'price_drops',
 'price_drops_yoy',
 'percent_active_listings_with_price_drops',
 'percent_active_listings_with_price_drops_yoy',
 'pending_sales',
 'pending_sales_yoy',
 'median_pending_sqft',
 'median_pending_sqft_yoy',
 'off_market_in_two_weeks',
 'off_market_in_two_weeks_yoy',
 'off_market_in_one_week',
 'off_market_in_one_week_yoy',
 'percent_off_market_in_two_weeks',
 'percent_off_market_i

In [8]:
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions, QueryOptions
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions, QueryOptions
from couchbase.bucket import Bucket
from couchbase.management.buckets import CreateBucketSettings

In [9]:
import os
import dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
housing_df.count()

4246231

In [23]:
couchbaseUsername, couchbasePassword = os.getenv('COUCHBASE_USERNAME'), os.getenv('COUCHBASE_PASSWORD')

# Get a reference to our cluster
auth = PasswordAuthenticator(couchbaseUsername, couchbasePassword)
cluster = Cluster.connect('couchbase://127.0.0.1', ClusterOptions(auth))

# Datasets structure: [(bucket_name, PySpark DataFrame)]
def db_populator(datasets, options=None):
    for bucketName, dataset in datasets:
        if options is not None:
            bucketSettings = CreateBucketSettings(
                name=options["name"], 
                bucket_type=options["bucket_type"], 
                ram_quota_mb=options["ram_quota_mb"], 
                max_expiry=options["max_expiry"], 
                compression_mode=options["compression_mode"], 
                conflict_resolution_type=options["conflict_resolution_type"]
            )
        else:
            bucketSettings = CreateBucketSettings(
                name=bucketName, 
                bucket_type="couchbase", 
                ram_quota_mb=1000, 
                max_expiry=0, 
                compression_mode="passive", 
                conflict_resolution_type="seqno"
            )
        
        print(f"Populating {bucketName} with documents")
        try: 
            cluster.buckets().create_bucket(bucketSettings)
            print('Bucket created')
            cluster.wait_until_ready(timedelta(seconds=20))
        except Exception as e:
            if "Bucket with given name already exists" in str(e):
                pass

        bucket = cluster.bucket(bucketName)
        collection = bucket.default_collection()

        # Use PySpark DataFrame API to collect rows and upsert into Couchbase
        for index, row in enumerate(dataset.rdd.toLocalIterator()):
            data = row.asDict()
            data["_id"] = str(index)  # Use the loop index as a unique identifier
            collection.upsert(data["_id"], data)

        print('Data stored')
        try:
            cluster.query_indexes().create_primary_index(bucketName)
            print('Index created')
        except Exception as e:
            if "already exists" in str(e):
                pass

In [24]:
db_populator([("housing", housing_df)])

Populating housing with documents
Bucket created


Data stored
Index created
